Use this utlity to update the returns and std_dev fields within investment-options.csv

In [9]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [10]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import brownbear as bb

# format price data
pd.options.display.float_format = '{:0.2f}'.format

%matplotlib inline

In [11]:
# set size of inline plots
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

Globals

In [12]:
# set refresh_timeseries=True to download timeseries.  Otherwise /symbol-cache is used.
refresh_timeseries = True

In [13]:
# read in sp400.csv
sp400 = pd.read_csv('sp400.csv')
sp400.drop(columns=['SEC filings'], inplace=True)
sp400.rename(columns={'Ticker symbol': 'Symbol',
                      'Security':'Description',
                      'GICS Sector':'Asset Class',
                      'GICS Sub-Industry': 'GICS Sub Industry'}, inplace=True)
sp400.set_index("Symbol", inplace=True)
sp400

,Description,Asset Class,GICS Sub Industry
Symbol,,,
ACHC,Acadia Healthcare,Health Care,Health Care Facilities
ACIW,ACI Worldwide,Information Technology,Application Software
ADNT,Adient plc,Consumer Discretionary,Auto Parts & Equipment
ATGE,Adtalem Global Education,Consumer Discretionary,Education Services
ACM,AECOM,Industrials,Construction & Engineering
...,...,...,...
XRX,Xerox,Information Technology,"Technology Hardware, Storage & Peripherals"
Y,Alleghany Corporation,Financials,Reinsurance
XPO,XPO Logistics,Industrials,Air Freight & Logistics


In [14]:
# read in gics-2-asset-class.csv
gics2asset_class = pd.read_csv('gics-2-asset-class.csv', skip_blank_lines=True, comment='#')
gics2asset_class.set_index("GICS", inplace=True)
gics2asset_class = gics2asset_class['Asset Class'].to_dict()
gics2asset_class

{'Energy': 'US Stocks:Energy',
 'Materials': 'US Stocks:Materials',
 'Industrials': 'US Stocks:Industrials',
 'Consumer Discretionary': 'US Stocks:Consumer Discretionary',
 'Consumer Staples': 'US Stocks:Consumer Staples',
 'Health Care': 'US Stocks:Healthcare',
 'Financials': 'US Stocks:Financials',
 'Information Technology': 'US Stocks:Technology',
 'Communication Services': 'US Stocks:Communication Services',
 'Utilities': 'US Stocks:Utilities',
 'Real Estate': 'US Stocks:Real Estate'}

In [15]:
# map sp400 GICS sectors to brownbear defined asset classes
def _asset_class(row):
    return gics2asset_class[row['Asset Class']]

sp400['Asset Class'] = sp400.apply(_asset_class, axis=1)

# yahoo finance uses '-' where '.' is used in symbol names
sp400.index = sp400.index.str.replace('.', '-')
sp400

,Description,Asset Class,GICS Sub Industry
Symbol,,,
ACHC,Acadia Healthcare,US Stocks:Healthcare,Health Care Facilities
ACIW,ACI Worldwide,US Stocks:Technology,Application Software
ADNT,Adient plc,US Stocks:Consumer Discretionary,Auto Parts & Equipment
ATGE,Adtalem Global Education,US Stocks:Consumer Discretionary,Education Services
ACM,AECOM,US Stocks:Industrials,Construction & Engineering
...,...,...,...
XRX,Xerox,US Stocks:Technology,"Technology Hardware, Storage & Peripherals"
Y,Alleghany Corporation,US Stocks:Financials,Reinsurance
XPO,XPO Logistics,US Stocks:Industrials,Air Freight & Logistics


In [16]:
# drop invalid symbols
#sp400.drop(['AAXN'], inplace=True)

In [17]:
# make symbols list
symbols = list(sp400.index)
#symbols

In [18]:
# get the timeseries for the symbols and compile into a single csv
bb.fetch_timeseries(symbols, refresh=refresh_timeseries)
bb.compile_timeseries(symbols)

................................................................................................................................................................................................................................................................................................................................................................................................................


In [19]:
# read symbols timeseries into a dataframe
df = pd.read_csv('symbols-timeseries.csv', skip_blank_lines=True, comment='#')
df.set_index("Date", inplace=True)
df = df[:]
df

,ACHC,ACIW,ADNT,ATGE,ACM,AMG,ADS,ACC,AEO,AFG,...,WW,WWD,WWE,WH,X,XRX,Y,XPO,YELP,YETI
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,59.98,19.71,nan,43.01,30.35,203.95,267.74,32.12,11.27,46.76,...,21.53,46.35,10.85,nan,25.10,28.13,444.64,40.63,55.15,nan
2015-01-05,59.12,19.26,nan,43.60,29.02,196.56,264.02,32.31,11.41,46.15,...,21.26,45.35,10.65,nan,23.93,27.52,435.03,39.37,52.53,nan
2015-01-06,58.19,18.95,nan,41.75,28.73,192.32,260.66,32.64,11.45,45.88,...,19.74,45.02,10.68,nan,23.20,27.15,432.47,37.98,52.44,nan
2015-01-07,60.63,19.03,nan,42.04,29.32,197.04,262.66,32.70,12.00,46.19,...,20.35,45.44,10.23,nan,23.26,27.50,434.10,38.14,52.21,nan
2015-01-08,61.76,19.01,nan,42.91,30.23,199.05,270.14,32.89,11.44,47.03,...,20.32,45.21,10.12,nan,23.77,28.25,437.79,38.35,53.83,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-28,64.36,38.26,50.06,36.38,65.01,164.00,121.05,47.16,35.43,133.06,...,39.30,127.18,55.84,75.06,25.93,23.45,716.57,146.93,40.11,87.60
2021-06-01,64.60,39.00,52.50,37.63,64.81,164.99,124.56,48.62,34.43,134.64,...,39.83,128.52,55.66,76.10,26.67,23.42,704.61,149.35,39.93,88.00
2021-06-02,64.03,38.82,50.62,37.36,64.61,166.30,124.72,48.77,33.67,136.48,...,40.00,126.72,56.01,74.70,26.12,24.04,701.45,146.46,39.47,87.30


In [20]:
# sample symbol
symbol = 'ACHC'

In [21]:
annual_returns = bb.annualize_returns(df, timeperiod='daily', years=1)
annual_returns[symbol]

127.46185203551272

In [22]:
# calculate annualized returns
annual_returns_1mo = bb.annualize_returns(df, timeperiod='daily', years=1/12)
annual_returns_3mo = bb.annualize_returns(df, timeperiod='daily', years=3/12)
annual_returns_1yr = bb.annualize_returns(df, timeperiod='daily', years=1)
annual_returns_3yr = bb.annualize_returns(df, timeperiod='daily', years=3)
annual_returns_5yr = bb.annualize_returns(df, timeperiod='daily', years=5)

In [23]:
# calculate volatility
daily_returns = df.pct_change()
years = bb.TRADING_DAYS_PER_MONTH / bb.TRADING_DAYS_PER_YEAR
vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years)
vola[symbol]

0.27051496365704825

In [24]:
# calculate downside volatility
ds_vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years, downside=True)
ds_vola[symbol]

0.15587555760167474

In [25]:
# resample df on a monthly basis
df.index = pd.to_datetime(df.index)
monthly = df.resample('M').ffill()
bb.print_full(monthly[symbol])

Date
2015-01-31   57.75
2015-02-28   63.23
2015-03-31   71.60
2015-04-30   68.50
2015-05-31   74.14
2015-06-30   78.33
2015-07-31   79.78
2015-08-31   73.03
2015-09-30   66.27
2015-10-31   61.41
2015-11-30   69.01
2015-12-31   62.46
2016-01-31   61.03
2016-02-29   55.41
2016-03-31   55.11
2016-04-30   63.19
2016-05-31   58.87
2016-06-30   55.40
2016-07-31   56.50
2016-08-31   51.19
2016-09-30   49.55
2016-10-31   35.96
2016-11-30   38.01
2016-12-31   33.10
2017-01-31   38.37
2017-02-28   44.72
2017-03-31   43.60
2017-04-30   43.58
2017-05-31   41.34
2017-06-30   49.38
2017-07-31   52.93
2017-08-31   46.94
2017-09-30   47.76
2017-10-31   31.36
2017-11-30   31.83
2017-12-31   32.63
2018-01-31   34.08
2018-02-28   38.10
2018-03-31   39.18
2018-04-30   35.58
2018-05-31   40.19
2018-06-30   40.91
2018-07-31   39.48
2018-08-31   41.53
2018-09-30   35.20
2018-10-31   41.50
2018-11-30   33.97
2018-12-31   25.71
2019-01-31   27.36
2019-02-28   26.29
2019-03-31   29.31
2019-04-30   32.02
2019-05

In [26]:
# calculate monthly returns
monthly_returns = monthly.pct_change()
monthly_returns[symbol]

Date
2015-01-31     nan
2015-02-28    0.09
2015-03-31    0.13
2015-04-30   -0.04
2015-05-31    0.08
              ... 
2021-02-28    0.09
2021-03-31    0.03
2021-04-30    0.07
2021-05-31    0.06
2021-06-30    0.02
Freq: M, Name: ACHC, Length: 78, dtype: float64

In [27]:
# calculate standard deviation
std_dev = bb.annualized_standard_deviation(monthly_returns, timeperiod='monthly', years=3)
std_dev[symbol]

0.4860420545101493

In [28]:
# read investment-options-header.csv
lines = []
with open('investment-options-in.csv', 'r') as f:
    lines = [line.strip() for line in f]
lines

['# Description: S&P 400 investment options. 10/02/2020',
 '',
 '# Format',
 '"Investment Option","Description","Asset Class","1 mo","3 mo","1 Yr","3 Yr","5 Yr","Vola","DS Vola","Std Dev"',
 '# Note: "Description" field is optional']

In [29]:
# for each symbol, write out the 1 Yr, 3 Yr, 5 Yr, and std dev
out = lines.copy()

# This is still slow (2.53 s)
for i, (index, row) in enumerate(sp400.iterrows()):

    symbol = index
    description = row['Description']
    asset_class = row['Asset Class']

    ret_1mo = annual_returns_1mo[symbol]
    ret_3mo = annual_returns_3mo[symbol]
    ret_1yr = annual_returns_1yr[symbol]
    ret_3yr = annual_returns_3yr[symbol]
    ret_5yr = annual_returns_5yr[symbol]
    
    if np.isnan(ret_3yr): ret_3yr = ret_1yr
    if np.isnan(ret_5yr): ret_5yr = ret_3yr

    _vola = vola[symbol]*100
    _ds_vola = ds_vola[symbol]*100
    sd = std_dev[symbol]*100

    out.append(
        '"{}","{}","{}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}"'
        .format(symbol, description, asset_class,
                ret_1mo, ret_3mo, ret_1yr, ret_3yr, ret_5yr, _vola, _ds_vola, sd)) 

In [30]:
# write out asset-classes.csv
with open('investment-options.csv', 'w') as f:
    for line in out:
        f.write(line + '\n')